# A ConcreteModel for CNLS estimation

+ Author: Sheng Dai (sheng.dai@aalto.fi)
+ Data  : Timo Kuosmanen (timo.kuosmanen@aalto.fi)
+ Date  : 2020-04-15

We also prepare a specific model that does not need to call any of our developed functions. In this concrete model, the users can give the parameter specification by reading data from an Excel file.

In [1]:
# Import of the pyomo module
import pandas 
import numpy
import sys
from pyomo.environ import *

# Creation of a Concrete Model
model = ConcreteModel(name = "CNLS")

In [2]:
# Sets
i = numpy.array(['i{}'.format(i) for i in range(1, 90)])
model.i = Set(initialize=i, doc='DMUs', ordered=True )

model.j = Set(initialize=['j1', 'j2', 'j3'], doc='inputs and outputs')

# alias
model.h = SetOf(model.i) 

In [3]:
# Parameters 
# output (y1,y2,y3)
df1 = pandas.read_excel('y.xlsx', header=0, index_col=0)
Dict1 = dict()
for i in df1.index:
    for j in df1.columns:
         Dict1[i, j] = float(df1[j][i])
model.y = Param(model.i, model.j, initialize = Dict1) 

# input (cost)
df2 = pandas.read_excel('x.xlsx', header=0, index_col=0)
Dict2 = dict()
for i in df2.index:
         Dict2[i] = float(df2['TOTEX'][i])
model.c = Param(model.i, initialize = Dict2) 

Note: Excel data set can be download from <https://github.com/ds2010/StoNED-Python/tree/master/Data>.

In [4]:
#Variables
model.b = Var(model.i, model.j, bounds=(0.0,None), doc='beta-coeff')
model.e = Var(model.i, doc='res')
model.f = Var(model.i, bounds=(0.0,None), doc='frontier')

# Constraints and objective f.
def qreg_rule(model, i):
    return log(model.c[i]) == log(model.f[i] + 1) + model.e[i]
model.qreg = Constraint(model.i, rule=qreg_rule, doc='log-transformed regression')

def qlog_rule(model, i):
    return model.f[i] == sum(model.b[i, j]*model.y[i, j] for j in model.j) - 1
model.qlog = Constraint(model.i, rule=qlog_rule, doc='cost function')

def qconcav_rule(model, i, h):
    return sum(model.b[i,j]*model.y[i,j] for j in model.j) >= sum(model.b[h,j]*model.y[i,j] for j in model.j)
model.qconcav = Constraint(model.i, model.h, rule=qconcav_rule, doc='concavity constraint')

def objective_rule(model):
  return sum(model.e[i]*model.e[i] for i in model.i)
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

In [5]:
#Solve the model
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [7]:
import numpy as np
#retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
df_b = [ i + tuple([j]) for i, j in zip(ind, val)]
df_b = np.asarray(df_b)

In [8]:
# display beta coefficients
df_b

array([['i1', 'j1', '8.33441576428776'],
       ['i1', 'j2', '0.998651257978137'],
       ['i1', 'j3', '0.010791032492034959'],
       ['i2', 'j1', '6.088027290570833e-08'],
       ['i2', 'j2', '0.9426507526186271'],
       ['i2', 'j3', '0.11508362781532097'],
       ['i3', 'j1', '5.014162094131568'],
       ['i3', 'j2', '0.9639507564922082'],
       ['i3', 'j3', '0.05782033576172141'],
       ['i4', 'j1', '6.928933052982218'],
       ['i4', 'j2', '1.108138444412587'],
       ['i4', 'j3', '0.007154119157112404'],
       ['i5', 'j1', '6.887181711023441'],
       ['i5', 'j2', '1.1397100685369213'],
       ['i5', 'j3', '6.203654231094351e-12'],
       ['i6', 'j1', '3.2283329698069827'],
       ['i6', 'j2', '0.8488801157314547'],
       ['i6', 'j3', '0.08990166557169409'],
       ['i7', 'j1', '6.887188452568707'],
       ['i7', 'j2', '1.1397096226181564'],
       ['i7', 'j3', '3.187200562454836e-08'],
       ['i8', 'j1', '8.334415909550087'],
       ['i8', 'j2', '0.9986511786734155'],
    

In [9]:
import pandas as pd
df_b = pd.DataFrame(df_b,columns = ['Name', 'Key', 'Value'])
df_b = df_b.pivot(index='Name', columns='Key', values='Value')

# retrive the residuals
ind = list(model.e)
val = list(model.e[:].value)
df_e = np.asarray(val)

In [10]:
#display residuals
df_e

array([ 0.03591705,  0.02603548,  0.20219662, -0.00451797,  0.17430052,
       -0.11912249, -0.01130134,  0.1192172 ,  0.37369844, -0.04018423,
        0.02303955, -0.07779539,  0.24790125,  0.25962007, -0.02513006,
       -0.14458385,  0.19495398,  0.16927398, -0.01384278, -0.09871993,
       -0.12696537, -0.26051316,  0.18701396, -0.09403671,  0.12152759,
        0.06493156,  0.24369437, -0.19088083,  0.10527361,  0.0277184 ,
       -0.00155868, -0.43113541, -0.0450978 ,  0.21595499, -0.00401978,
        0.15996073, -0.21593312, -0.15266743, -0.115049  , -0.03704671,
       -0.04825153,  0.04513426,  0.06501279, -0.06184066, -0.08923651,
       -0.13083339,  0.02732435,  0.08403226, -0.13731314, -0.13258987,
        0.19606224, -0.02272717,  0.20040745,  0.1998938 ,  0.04287153,
       -0.21797092, -0.22973008, -0.08861547, -0.19033982,  0.02023355,
       -0.21417276, -0.08037898, -0.03575945,  0.01552399,  0.36593916,
       -0.10950723,  0.08123884, -0.04885468,  0.08217855, -0.23